In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

# Replace 'America/Chicago' with your correct time zone
now = datetime.now(ZoneInfo("America/Chicago"))
print("Local Date and Time:", now.strftime("%Y-%m-%d %H:%M:%S"))


Local Date and Time: 2025-05-04 14:56:03


In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import random

# Load models
models = {
    '3_hours': load_model('3_hours_ahead.keras'),
    '6_hours': load_model('6_hours_ahead.keras'),
    '9_hours': load_model('9_hours_ahead.keras'),
    '12_hours': load_model('12_hours_ahead.keras'),
    '15_hours': load_model('15_hours_ahead.keras'),
    '18_hours': load_model('18_hours_ahead.keras'),
    '21_hours': load_model('21_hours_ahead.keras'),
    '24_hours': load_model('24_hours_ahead.keras')
}

weather_info = {
    'clear': {'label': 'Clear', 'emoji': '☀️', 'night_emoji': '🌙'},
    'rain': {'label': 'Rain', 'emoji': '🌧️', 'night_emoji': '🌧️'},
    'reduced_visibility': {'label': 'Fog', 'emoji': '🌫️', 'night_emoji': '🌫️'},
    'snow': {'label': 'Snow', 'emoji': '❄️', 'night_emoji': '❄️'}
}

def predict_weather(hours_ahead):
    # Mock prediction function
    try:
        model = models[f'{hours_ahead}_hours']
        dummy_input = np.random.randn(1, 40, model.input_shape[2])
        prediction = model.predict(dummy_input)
        predicted_class = np.argmax(prediction, axis=1)[0]
        weather = list(weather_info.keys())[predicted_class]

        forecast_time = datetime.now(ZoneInfo("America/Chicago")) + timedelta(hours=hours_ahead)
        time_str = forecast_time.strftime("%-I %p").lower()
        is_daytime = 6 <= forecast_time.hour < 18

        if is_daytime:
            temp = random.randint(65, 85)
            feels_like = temp + random.randint(-3, 3)
            weather_emoji = weather_info[weather]['emoji']
        else:
            temp = random.randint(50, 70)
            feels_like = temp + random.randint(-5, 0)
            weather_emoji = weather_info[weather]['night_emoji']

        precip = random.randint(0, 100) if weather in ['rain', 'snow'] else 0

        return {
            'time': time_str,
            'day': forecast_time.strftime("%a"),
            'weather': weather_emoji,
            'weather_label': weather_info[weather]['label'],
            'temp': f"{temp}°",
            'feels_like': f"Feels {feels_like}°",
            'precip': f"{precip}%" if precip > 0 else "",
            'is_daytime': is_daytime,
            'hour': forecast_time.hour
        }

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def get_all_predictions():
    """Get predictions for all time intervals"""
    return [predict_weather(h) for h in range(3, 25, 3) if predict_weather(h)]

def create_forecast_app():
    with gr.Blocks(title="Weather Forecast", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🌤️ 24-Hour Weather Forecast
        *Arlington, TX | Updated: {time}*
        """.format(time=datetime.now(ZoneInfo("America/Chicago")).strftime("%a %-I:%M %p")))

        # Create forecast rows
        with gr.Group():
            for pred in get_all_predictions():
                time_of_day = "day" if pred['is_daytime'] else "night"
                with gr.Row(variant="panel", elem_classes=f"forecast-row {time_of_day}"):
                    # Time column
                    with gr.Column(min_width=100):
                        gr.Markdown(f"""
                        ### {pred['day']}
                        {pred['time']}
                        """)

                    # Weather column
                    with gr.Column(min_width=100):
                        gr.Markdown(f"""
                        <div style="font-size: 24px; text-align: center">{pred['weather']}</div>
                        <div style="text-align: center">{pred['weather_label']}</div>
                        """)

                    # Temp column
                    with gr.Column(min_width=100):
                        gr.Markdown(f"""
                        <div style="font-size: 24px; text-align: center">{pred['temp']}</div>
                        <div style="text-align: center; font-size: 0.9em">{pred['feels_like']}</div>
                        """)

                    # Precip column
                    with gr.Column(min_width=80):
                        if pred['precip']:
                            gr.Markdown(f"""
                            <div style="text-align: center; font-size: 0.9em">Precip</div>
                            <div style="text-align: center">{pred['precip']}</div>
                            """)

        # Add some CSS styling
        demo.css = """
        .forecast-row {
            border-radius: 12px;
            padding: 12px;
            margin-bottom: 8px;
            border: 1px solid #e0e0e0;
        }
        .forecast-row.day {
            background: linear-gradient(135deg, #0f2027 0%, #203a43 50%, #2c5364 100%);
            color: white;
        }
        .forecast-row.night {
            background: #000000;
            color: white;
        }
        .forecast-row.night * {
            color: white;
        }
        .forecast-row.day * {
            color: white;
        }
        """
    return demo

# Launch the app
app = create_forecast_app()
app.launch()

ModuleNotFoundError: No module named 'gradio'

In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import random

# Load models
models = {
    '3_hours': load_model('3_hours_ahead.keras'),
    '6_hours': load_model('6_hours_ahead.keras'),
    '9_hours': load_model('9_hours_ahead.keras'),
    '12_hours': load_model('12_hours_ahead.keras'),
    '15_hours': load_model('15_hours_ahead.keras'),
    '18_hours': load_model('18_hours_ahead.keras'),
    '21_hours': load_model('21_hours_ahead.keras'),
    '24_hours': load_model('24_hours_ahead.keras')
}

weather_info = {
    'clear': {'label': 'Sunny', 'emoji': '☀️'},
    'rain': {'label': 'Rainy', 'emoji': '🌧️'},
    'reduced_visibility': {'label': 'Foggy', 'emoji': '🌫️'},
    'snow': {'label': 'Snowy', 'emoji': '❄️'}
}

def predict_weather(hours_ahead):
    #Mock prediction function
    try:
        model = models[f'{hours_ahead}_hours']
        # dummy_input = np.random.randn(1, 40, model.input_shape[2])
        prediction = model.predict(dummy_input)
        predicted_class = np.argmax(prediction, axis=1)[0]
        weather = list(weather_info.keys())[predicted_class]

        forecast_time = datetime.now(ZoneInfo("America/Chicago")) + timedelta(hours=hours_ahead)
        time_str = forecast_time.strftime("%-I %p").lower()
        is_daytime = 6 <= forecast_time.hour < 18

        if is_daytime:
            temp = random.randint(65, 85)
            feels_like = temp + random.randint(-3, 3)
        else:
            temp = random.randint(50, 70)
            feels_like = temp + random.randint(-5, 0)

        precip = random.randint(0, 100) if weather in ['rain', 'snow'] else 0

        return {
            'time': time_str,
            'day': forecast_time.strftime("%a"),
            'weather': weather_info[weather]['emoji'],
            'temp': f"{temp}°",
            'feels_like': f"Feels {feels_like}°",
            'precip': f"{precip}%" if precip > 0 else "",
            'is_daytime': is_daytime
        }

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def get_all_predictions():
    """Get predictions for all time intervals"""
    return [predict_weather(h) for h in range(3, 25, 3) if predict_weather(h)]

def create_forecast_app():
    with gr.Blocks(title="Weather Forecast") as demo:
        gr.Markdown("# 🌤️ 24-Hour Weather Forecast")

        # Create forecast rows
        forecast_rows = []
        predictions = get_all_predictions()

        for pred in predictions:
            with gr.Row():
                # Time column
                with gr.Column(min_width=80):
                    gr.Markdown(f"**{pred['day']} {pred['time']}**")

                # Weather column
                with gr.Column(min_width=60):
                    gr.Markdown(f"**{pred['weather']}**")

                # Temp column
                with gr.Column(min_width=60):
                    gr.Markdown(f"**{pred['temp']}**")

                # Feels like column
                with gr.Column(min_width=120):
                    gr.Markdown(f"*{pred['feels_like']}*")

                # Precip column
                with gr.Column(min_width=60):
                    if pred['precip']:
                        gr.Markdown(f"**{pred['precip']}**")
    return demo

# Launch the app
app = create_forecast_app()
app.launch()